## Libraries

In [1]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report, recall_score, accuracy_score,roc_curve, auc, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import spacy
from spacy.cli import download
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from joblib import dump, load
from google.colab import drive
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kde
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Asegurarse de que NLTK esté instalado y descargue los recursos necesarios
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
# Cargar el modelo de lenguaje preentrenado en español
download('es_core_news_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


## Cargar datasets



In [3]:
import pandas as pd

## Dataset con 2 dígitos
''' df = pd.read_csv('https://raw.githubusercontent.com/mnavasloro/CPVClassifier/main/data/dataset/train.csv')
dftest = pd.read_csv('https://raw.githubusercontent.com/mnavasloro/CPVClassifier/main/data/dataset/test.csv')

df.pop('Unnamed: 0')
dftest.pop('Unnamed: 0') '''

#Dataset con 3 dígitos
# Leer los datasets
''' df = pd.read_csv('https://raw.githubusercontent.com/mnavasloro/CPVClassifier/main/data/data3/3digits_train.csv')
dftest = pd.read_csv('https://raw.githubusercontent.com/mnavasloro/CPVClassifier/main/data/data3/3digits_test.csv') '''
df = pd.read_csv("ruta")
dftest = pd.read_csv("ruta")

df.pop('cpv')
#dftest.pop('CPV')

df.head()

,description,030,031,032,033,034,090,091,092,093,...,924,925,926,927,980,981,982,983,985,989
0,Contratación de la ruta 3701726 de transporte ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Servicio de montaje y desmontaje de la exposic...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Prestación de los servicios de realización, pr...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Canalizaciones fibra óptica / telecomunicacion...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acuerdo Marco de suministro de combustible gas...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Prueba individual de 2 dígitos

In [4]:
twodig = '60'
columns = df.columns.str.startswith(twodig) #Cogemos las columnas que empiezan por los dos primeros digitos que queremos

In [5]:
columns[0]= True #También queremos la descripcion, asi que la primera columna tambien la ponemos a True. Para el otro dataset columns[0]
df_03 = df.loc[:,columns] #Nos quedamos con las columnas seleccionadas
df_03 = df_03[df_03[list(df_03.columns[1:].values)].any(axis='columns')] #Nos quedamos solo las filas que tengan algún 1

In [6]:
df_03.head()

,description,600,601,602,603,604,605,606
0,Contratación de la ruta 3701726 de transporte ...,0,1,0,0,0,0,0
12,Servicios de envíos de paquetería,0,1,0,0,0,0,0
69,Ruta transporte escolar CC098. Categoría A.,0,1,0,0,0,0,0
114,Contratación de Conducción Extraordinaria de T...,0,1,0,0,0,0,0
257,supersimplificado ruta 225 S toledo curso 2021...,0,1,0,0,0,0,0


In [ ]:
#Saber cuantos 1 tiene alguna clase
df_03["603"].value_counts()

In [7]:
df.iloc[114].description

'Contratación de Conducción Extraordinaria de Transporte del Correo en La Zona 4, Madrid Uses Apoyo a Centro y Unidades (6 lotes).'

In [8]:
# Crear una lista para almacenar las descripciones y los dos primeros dígitos de las columnas numéricas
X = []
# Crear una lista para almacenar el tercer dígito de las columnas numéricas
y = []

# Iterar sobre las filas del DataFrame
for index, row in df_03.iterrows():
    descripcion = row['description']

    # Iterar sobre las columnas numéricas (ignorar la columna 'descripcion')
    for col in df_03.columns[1:]:
        if row[col] == 1:
            two_digits = col[:2]
            third_digit = col[2]

            # Almacenar la descripción y los dos primeros dígitos en X
            X.append((descripcion, two_digits))

            # Almacenar el tercer dígito en y
            y.append(third_digit)

In [9]:
# Convertir las listas en DataFrames
X = pd.DataFrame(X, columns=['description', 'two_digits'])
y = pd.DataFrame(y, columns=['third_digit'])

In [10]:
# Cargar el modelo en español de spaCy
nlp = spacy.load('es_core_news_sm')
# Función para procesar las descripciones utilizando técnicas de NLP
def preprocess_text(text):
    # Lowercasing, procesar el texto y crear un objeto Doc
    doc = nlp(text.lower())

    # Noise removal and lemmatization
    words = [token.lemma_ for token in doc if not token.is_punct]

    # Stop word removal
    words = [word for word in words if word not in nlp.Defaults.stop_words]

    # Rejoin words
    text = ' '.join(words)

    return text


In [11]:
# Aplicar la función de preprocesamiento a cada descripción en el DataFrame
X['description'] = X['description'].apply(preprocess_text)

In [12]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Crear un vectorizador TF-IDF para transformar la descripción
#vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
vectorizer = TfidfVectorizer(max_features=1000)

# Entrenar el vectorizador en los datos de entrenamiento
X_train_tfidf = vectorizer.fit_transform(X_train['description'])

# Aplicar el vectorizador a los datos de prueba
X_test_tfidf = vectorizer.transform(X_test['description'])

In [14]:
#Prueba con Random Forest
clf = RandomForestClassifier(random_state=42)

#Prueba con Decision Tree
#clf = DecisionTreeClassifier(random_state=42)

# Crear un clasificador Naive Bayes
#clf = MultinomialNB()

# Crear un clasificador k-vecinos más cercanos
#clf = KNeighborsClassifier(n_neighbors=3)

# Crear un clasificador Gradient Boosting
#clf = GradientBoostingClassifier(random_state=42)

#Prueba con SVM
#clf = SVC(random_state=42)

# Crear un pipeline para el clasificador
pipeline = Pipeline([
    ('clf', clf)
])

#------------Hiperparametros------------

#------------Para RandomForest----------
# Definir los parámetros a buscar
param_dist = {
        "clf__n_estimators": [100, 200, 300, 400, 500],
        "clf__max_depth": [10, 20, 30, 40, 50, None],
        "clf__min_samples_split": [2, 5, 10, 15],
        "clf__min_samples_leaf": [1, 2, 5, 10],
    }

# Inicializa RandomizedSearchCV
random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_iter=10,
        cv=3,
        scoring="accuracy",
        random_state=42,
)
#-----------Para DecisionTree-----------
# Definir los hiperparámetros
''' param_dist = {
    'clf__criterion': ['gini', 'entropy'],
    'clf__max_depth': np.arange(1, 20),
    'clf__min_samples_split': np.arange(2, 20),
    'clf__min_samples_leaf': np.arange(1, 20),
}

# Inicializa RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=100,
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
) '''

#-----------Para MultinominalNB-----------
# Define los hiperparámetros
''' param_dist = {
    'clf__alpha': np.linspace(0.5, 1.5, 6),
    'clf__fit_prior': [True, False],
}

# Inicializa RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=100,
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
) '''

#-----------Para GradientBoosting----------
# Define los hiperparámetros y las distribuciones para muestrear
''' param_dist = {
    'clf__n_estimators': [100, 200, 300, 400, 500],
    'clf__learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
    'clf__max_depth': np.arange(1, 10),
    'clf__min_samples_split': np.arange(2, 10),
    'clf__min_samples_leaf': np.arange(1, 10),
}

# Inicializa RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=100,
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
) '''

#-----------Para SVM----------
# Definir los hiperparámetros
'''param_dist = {
    'clf__C': [0.1, 1, 10, 100],
    'clf__gamma': ['scale', 'auto'],
    'clf__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'clf__degree': [2, 3, 4]  # Ignorado por todos los kernels excepto 'poly'
}

# Inicializar RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)'''



"param_dist = {\n    'clf__C': [0.1, 1, 10, 100],\n    'clf__gamma': ['scale', 'auto'],\n    'clf__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],\n    'clf__degree': [2, 3, 4]  # Ignorado por todos los kernels excepto 'poly'\n}\n\n# Inicializar RandomizedSearchCV\nrandom_search = RandomizedSearchCV(\n    pipeline,\n    param_distributions=param_dist,\n    n_iter=10,\n    cv=3,\n    verbose=1,\n    random_state=42,\n    n_jobs=-1\n)"

In [15]:
# Entrenar el modelo
random_search.fit(X_train_tfidf, y_train.values.ravel())

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('clf',
                                              RandomForestClassifier(random_state=42))]),
                   param_distributions={'clf__max_depth': [10, 20, 30, 40, 50,
                                                           None],
                                        'clf__min_samples_leaf': [1, 2, 5, 10],
                                        'clf__min_samples_split': [2, 5, 10,
                                                                   15],
                                        'clf__n_estimators': [100, 200, 300,
                                                              400, 500]},
                   random_state=42, scoring='accuracy')

In [16]:
# Predecir usando el conjunto de prueba
y_pred = random_search.predict(X_test_tfidf)

In [17]:
# Evaluar el rendimiento del modelo
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.30      0.43       267
           1       0.86      0.98      0.92      1397
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.88      0.64      0.74        58
           5       0.00      0.00      0.00         1
           6       0.81      0.62      0.70        34

    accuracy                           0.86      1760
   macro avg       0.47      0.36      0.40      1760
weighted avg       0.85      0.86      0.83      1760

Accuracy: 0.8579545454545454


In [ ]:
descripcion_nueva = "Contratación de Conducción Extraordinaria de Transporte del Correo en La Zona 4, Madrid Uses Apoyo a Centro y Unidades (6 lotes)."
descripcion_vectorizada = vectorizer.transform([descripcion_nueva])

In [ ]:
third_digit_pred = random_search.predict(descripcion_vectorizada)
print(f"El tercer dígito predicho es: {twodig}{third_digit_pred[0]}")

El tercer dígito predicho es: 601


In [ ]:
#Matriz de confusión
# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Visualizar la matriz de confusión
plt.figure(figsize=(7, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
#plt.title('Matriz de Confusión')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

## Exploración de los datos

In [ ]:
df_03.head()

,description,800,801,802,803,804,805,806
43,Servicio de formación en Sensibilización sobre...,0,0,0,0,0,1,0
76,Jornada de convivencia infantil Micropolix,1,0,0,0,0,0,0
81,"Contrato de servicios docentes externos, para ...",0,0,0,0,0,1,0
87,Contrato derivado del acuerdo marco para la co...,0,0,0,0,0,1,0
156,Prórroga del servicio de aeronaves de ala rota...,0,0,0,0,1,0,0


In [ ]:
#df.info()
#df.isnull().sum()
# Filtrar por aquellos valores de 3 dígitos que contengan 1
values = {'800': df_03[df_03["800"]==1].shape[0], '801': df_03[df_03["801"]==1].shape[0], '802': df_03[df_03["802"]==1].shape[0], '803': df_03[df_03["803"]==1].shape[0],
           '804': df_03[df_03["804"]==1].shape[0],  '805': df_03[df_03["805"]==1].shape[0], '806': df_03[df_03["806"]==1].shape[0]}

plt.bar(values.keys(), values.values())
#plt.title('Counts of 1s in each column')
plt.xlabel('Grupos')
plt.ylabel('Cantidad de Observaciones')
plt.show()

''' #Filtrar valores por aquellos valores de 2 dígitos que contengan 1
# Crea un diccionario para almacenar las cuentas de los '1's para cada columna
counts = {column: df[column].value_counts().get(1, 0) for column in df.columns}

# Convierte el diccionario a una serie de pandas para facilitar el gráfico
counts_series = pd.Series(counts)

# Crea el gráfico de barras
plt.figure(figsize=(10,6))
counts_series.plot(kind='bar')
plt.xlabel('Grupos')
plt.ylabel('Cantidad de observaciones')
plt.show() '''


### Saber la clase con menos observaciones y desbalanceada

In [ ]:
#Saber la clase con menos observaciones

df_temp = df.drop(columns=['description'])
# Esto asume que estás buscando la columna con menos 1s
min_ones = min(df_temp[df_temp == 1].count())

# Ahora busca la columna que tiene esa cantidad de 1s
min_ones_column = df_temp.columns[(df_temp == 1).sum() == min_ones][0]

print("La columna con menos unos es:", min_ones_column)

La columna con menos unos es: 428


In [ ]:
#Saber cuál es la clase más desbalanceada

# inicializa un diccionario para almacenar los desbalanceos
desbalanceo = {}

for columna in df_temp.columns:
    # calcula el desbalanceo en esta columna
    counts = df_temp[columna].value_counts()
    porcentaje_clase_minoritaria = min(counts) / sum(counts) * 100
    desbalanceo[columna] = porcentaje_clase_minoritaria

# encuentra la clase más desbalanceada
clase_mas_desbalanceada = min(desbalanceo, key=desbalanceo.get)

print(f"La clase más desbalanceada es {clase_mas_desbalanceada} con un porcentaje de {desbalanceo[clase_mas_desbalanceada]}% para la clase minoritaria.")


La clase más desbalanceada es 428 con un porcentaje de 0.0002592957527355702% para la clase minoritaria.


In [ ]:
#Saber cuál es la clase más balanceada

# Se supone que `df` es tu DataFrame y que todas sus columnas representan clases distintas.

# inicializa un diccionario para almacenar los desbalanceos
desbalanceo = {}

for columna in df_temp.columns:
    # calcula el desbalanceo en esta columna
    counts = df_temp[columna].value_counts()
    porcentaje_clase_minoritaria = min(counts) / sum(counts) * 100
    desbalanceo[columna] = porcentaje_clase_minoritaria

# encuentra la clase más balanceada
clase_mas_balanceada = max(desbalanceo, key=desbalanceo.get)

print(f"La clase más balanceada es {clase_mas_balanceada} con un porcentaje de {desbalanceo[clase_mas_balanceada]}% para la clase minoritaria.")

La clase más balanceada es 452 con un porcentaje de 14.854275786962608% para la clase minoritaria.


## Prueba con todos los 2 primeros dígitos

### Código para quedarnos sólo las filas y columnas cuyo código empieza por dos dígitos específico

In [ ]:
twodig = '38'
columns = df.columns.str.startswith(twodig) #Cogemos las columnas que empiezan por los dos primeros digitos que queremos

In [ ]:
columns[1]= True #También queremos la descripcion, asi que la primera columna tambien la ponemos a True
df_03 = df.loc[:,columns] #Nos quedamos con las columnas seleccionadas
df_03 = df_03[df_03[list(df_03.columns[1:].values)].any(axis='columns')] #Nos quedamos solo las filas que tengan algún 1

In [ ]:
df_03.head(20)

In [ ]:
df.iloc[216].description

'Quinto pedido derivado del Acuerdo Marco para el suministro de equipamiento multimedia. (Exp. 066/20).'

### Función para procesar las descripciones

In [ ]:
# Cargar el modelo en español de spaCy
nlp = spacy.load('es_core_news_sm')
# Función para procesar las descripciones utilizando técnicas de NLP
def preprocess_text(text):
    # Lowercasing, procesar el texto y crear un objeto Doc
    doc = nlp(text.lower())

    # Noise removal and lemmatization
    words = [token.lemma_ for token in doc if not token.is_punct]

    # Stop word removal
    words = [word for word in words if word not in nlp.Defaults.stop_words]

    # Rejoin words
    text = ' '.join(words)

    return text

### Entrenamiento del modelo

In [ ]:
from collections import defaultdict
two_digits_list = ['03', '09', '14', '15', '16', '18', '19', '22', '24', '30', '31', '32', '33', '34', '35', '37', '38', '39', '41', '42', '43', '44', '45', '48', '50', '51', '55', '60', '63', '64', '65', '66', '70', '71', '72', '73','75', '76', '77', '79', '80', '85', '90', '92', '98']
# Inicializar un diccionario para almacenar los resultados
results = {twodig: {} for twodig in two_digits_list}
path = "ruta"
def train_and_evaluate_for_twodig(twodig):
    columns = df.columns.str.startswith(twodig)
    columns[1] = True #columns[1] = True
    df_twodig = df.loc[:, columns]
    df_twodig = df_twodig[df_twodig[list(df_twodig.columns[1:].values)].any(axis='columns')]

    X = []
    y = []

    for index, row in df_twodig.iterrows():
        descripcion = row['description']

        for col in df_twodig.columns[1:]:
            if row[col] == 1:
                two_digits = col[:2]
                third_digit = col[2]

                X.append((descripcion, two_digits))
                y.append(third_digit)

    X = pd.DataFrame(X, columns=['description', 'two_digits'])
    y = pd.DataFrame(y, columns=['third_digit'])

    X['description'] = X['description'].apply(preprocess_text)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    vectorizer = TfidfVectorizer(max_features=1000)
    X_train_tfidf = vectorizer.fit_transform(X_train['description'])
    X_test_tfidf = vectorizer.transform(X_test['description'])

  #---------Random Forest---------

    rf_model = RandomForestClassifier(random_state=42)

    pipeline = Pipeline([
        ('clf', rf_model)
    ])


    # Especifica los rangos de hiperparámetros
    param_dist = {
        "clf__n_estimators": [100, 200, 300, 400, 500],
        "clf__max_depth": [10, 20, 30, 40, 50, None],
        "clf__min_samples_split": [2, 5, 10, 15],
        "clf__min_samples_leaf": [1, 2, 5, 10],
    }

    # Inicializa RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_dist,
        n_iter=10,
        cv=3,
        scoring="accuracy",
        random_state=42,
    )

    # Ajusta RandomizedSearchCV en los datos de entrenamiento
    random_search.fit(X_train_tfidf, y_train.values.ravel())

    # Obtén el mejor pipeline y sus hiperparámetros
    best_pipeline = random_search.best_estimator_
    best_params = random_search.best_params_

    y_pred = best_pipeline.predict(X_test_tfidf)

    accuracy = accuracy_score(y_test, y_pred)
   
    #Guardar el pipeline y el vectorizer
    dump(best_pipeline, f'{path}pipeline_{twodig}.joblib')
    dump(vectorizer, f'{path}vectorizer_{twodig}.joblib')

    return accuracy, pipeline, vectorizer

In [ ]:
# Entrenar y evaluar un modelo para cada par de dígitos
for twodig in two_digits_list:
    accuracy, model, vectorizer = train_and_evaluate_for_twodig(twodig)
    results[twodig]['accuracy'] = accuracy
    results[twodig]['model'] = model
    results[twodig]['vectorizer'] = vectorizer

In [ ]:
# Dada la descripción, predecir el mejor tercer dígito
def predict_best_third_digit(description, twodigits):
    # Cargar el modelo y el vectorizer
    model = load(f'{path}pipeline_{twodig}.joblib')
    vectorizer = load(f'{path}vectorizer_{twodig}.joblib')

    # Transformar la descripción con el vectorizer cargado
    descripcion_vectorizada = vectorizer.transform([description])

    # Predecir la probabilidad y la etiqueta
    third_digit_proba = model.predict_proba(descripcion_vectorizada)
    third_digit_pred = model.predict(descripcion_vectorizada)

    return third_digit_pred[0], np.max(third_digit_proba)

In [ ]:
descripcion_nueva = df.iloc[216].description
twodig = '38'
best_prediction, best_proba = predict_best_third_digit(descripcion_nueva,twodig)
print(f"La mejor predicción para la descripción dada es:")
print(f"Dos dígitos: {twodig}, tercer dígito: {best_prediction}, probabilidad: {best_proba}")
print(f"Concatenación de los 3 dígitos: {twodig}{best_prediction}")

La mejor predicción para la descripción dada es:
Dos dígitos: 38, tercer dígito: 6, probabilidad: 0.6402952822890903
Concatenación de los 3 dígitos: 386
